In [1]:
import random
import sys
from ImportantConfig import Config
config = Config()
from sql2fea import TreeBuilder,value_extractor
from NET import TreeNet
from sql2fea import Sql2Vec
from TreeLSTM import SPINN
import pandas as pd
import json
from IPython.display import display

sys.stdout = open(config.log_file, "w")
random.seed(113)
with open(config.queries_file) as f:
    import json
    queries = json.load(f)

tree_builder = TreeBuilder()
sql2vec = Sql2Vec()

In [2]:

df=pd.read_csv('./queries.csv')
df.head()

,query_id,query
0,1,"select channel, col_name, d_year, d_qoy, i_ca..."
1,2,"select i_item_id, \n avg(ss_quantity) ..."
2,3,"select sum (ss_quantity)\n from store_sales, s..."
3,4,"select i_item_id\n ,i_item_desc \n ..."
4,5,with ss_items as\n (select i_item_id item_id\n...


In [3]:
queries=df['query'].values
queries[0]

"select  channel, col_name, d_year, d_qoy, i_category, COUNT(*) sales_cnt, SUM(ext_sales_price) sales_amt FROM (\n        SELECT 'store' as channel, 'ss_hdemo_sk' col_name, d_year, d_qoy, i_category, ss_ext_sales_price ext_sales_price\n         FROM store_sales, item, date_dim\n         WHERE ss_hdemo_sk IS NULL\n           AND ss_sold_date_sk=d_date_sk\n           AND ss_item_sk=i_item_sk\n        UNION ALL\n        SELECT 'web' as channel, 'ws_ship_hdemo_sk' col_name, d_year, d_qoy, i_category, ws_ext_sales_price ext_sales_price\n         FROM web_sales, item, date_dim\n         WHERE ws_ship_hdemo_sk IS NULL\n           AND ws_sold_date_sk=d_date_sk\n           AND ws_item_sk=i_item_sk\n        UNION ALL\n        SELECT 'catalog' as channel, 'cs_ship_addr_sk' col_name, d_year, d_qoy, i_category, cs_ext_sales_price ext_sales_price\n         FROM catalog_sales, item, date_dim\n         WHERE cs_ship_addr_sk IS NULL\n           AND cs_sold_date_sk=d_date_sk\n           AND cs_item_sk=i

In [16]:
from PGUtils import pgrunner
plan_jsons=[]
for sql in queries:
    plan_jsons.append(pgrunner.getCostPlanJson(sql))
len(plan_jsons)

17813

In [19]:
error_count = 0
queryid_final=[]
for query_id,plan in zip(list(range(1,17814)),plan_jsons):
    try:
        tree_builder.plan_to_feature_tree(plan)
        queryid_final.append(query_id)
    except Exception as e:
        error_count += 1
        print(f"Error in processing plan: {e}")

display(f"too much children Total errors: {error_count}")

'Total errors: 3413'

In [31]:
filtered_df = df[df['query_id'].isin(queryid_final)]
queryid_final

[2,
 3,
 4,
 5,
 6,
 8,
 9,
 11,
 12,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 29,
 31,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 45,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 99,
 100,
 102,
 103,
 104,
 106,
 107,
 108,
 109,
 111,
 112,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 141,
 142,
 143,
 144,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 154,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 168,
 170,
 171,
 172,
 173,
 174,
 177,
 179,
 181,
 182,
 183,
 186,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 199,
 200,
 202,
 203,
 205,
 207,
 208,
 209,
 211,
 212,
 213,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,

In [26]:
filtered_df=filtered_df.drop(columns=['query_id'])
filtered_df.head()

,query
1,"select i_item_id, \n avg(ss_quantity) ..."
2,"select sum (ss_quantity)\n from store_sales, s..."
3,"select i_item_id\n ,i_item_desc \n ..."
4,with ss_items as\n (select i_item_id item_id\n...
5,"select asceding.rnk, i1.i_product_name best_p..."


In [27]:
filtered_df.to_csv("./information/queries_filtered.csv")

In [28]:
total=['Limit', 'Aggregate', 'Gather Merge', 'Sort', 'Append', 'Hash Join', 'Nested Loop', 'Hash', 'Seq Scan', 'Index Scan', 'Gather', 'WindowAgg', 'Merge Join', 'Materialize', 'Subquery Scan', 'CTE Scan', 'Group', 'Index Only Scan', 'Bitmap Heap Scan', 'Bitmap Index Scan', 'SetOp', 'Result', 'Unique', 'Merge Append']
after_fillter=['Limit', 'Aggregate', 'Gather Merge', 'Sort', 'Nested Loop', 'Index Scan', 'Hash Join', 'Hash', 'Seq Scan', 'Gather', 'WindowAgg', 'Merge Join', 'Materialize', 'Subquery Scan', 'Group', 'Index Only Scan', 'CTE Scan', 'SetOp', 'Append', 'Result', 'Unique', 'Bitmap Heap Scan', 'Bitmap Index Scan']
set(total)-set(after_fillter)

{'Merge Append'}

In [10]:
limit_operators = ['Limit']
aggregate_operators = ['Aggregate', 'Group', 'WindowAgg']
merge_and_join_operators = ['Gather Merge', 'Merge Join', 'Nested Loop', 'Hash Join']
sort_and_scan_operators = ['Sort', 'Seq Scan', 'Index Scan', 'Index Only Scan', 'Bitmap Heap Scan', 'Bitmap Index Scan']
gather_and_materialize_operators = ['Gather', 'Materialize']
subquery_and_cte_operators = ['Subquery Scan', 'CTE Scan']
set_operation_and_others = ['SetOp', 'Append', 'Result', 'Unique']


In [11]:
set(total)-set(scan)-set(join)

{'Aggregate',
 'Append',
 'Gather',
 'Gather Merge',
 'Group',
 'Hash',
 'Limit',
 'Materialize',
 'Merge Append',
 'Result',
 'SetOp',
 'Sort',
 'Unique',
 'WindowAgg'}